# Segmenting and Clustering Neighborhoods in Toronto

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

## 1 Import generic modules

In [1]:
import pandas as pd
import numpy as np

## 2 Scrape Neighbourhood data from Wikipedia

#### Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

##### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [105]:
dfNeighborhoods = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

Data scraped using Beautiful Soup having looked at several posts:

* Using instructions from https://simpleanalytical.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas
* See also instructions from https://stackoverflow.com/questions/50355577/scraping-wikipedia-tables-with-python-selectively
* See also instructions from https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722

In [106]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

request = requests.get(URL).text
soup = BeautifulSoup(request,'lxml')

myTable = soup.find('table',{'class':'wikitable sortable'})

postalCode=[]
borough=[]
neighborhood=[]

for row in myTable.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postalCode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))
        
dfNeighborhoods['PostalCode'] = postalCode
dfNeighborhoods['Borough'] = borough
dfNeighborhoods['Neighborhood'] = neighborhood

dfNeighborhoods = dfNeighborhoods.replace(r'\n','', regex=True) # remove the \n new line characters

dfNeighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

Removed the Not assigned boroughs using simple slicing

In [107]:
dfNeighborhoods = dfNeighborhoods[dfNeighborhoods['Borough'] != 'Not assigned']

dfNeighborhoods.reset_index(drop=True, inplace=True)

dfNeighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

This seemed to happen without any further action required. See for example:

In [108]:
dfNeighborhoods[dfNeighborhoods['PostalCode']=='M5A']

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

There aren't any rows where the neighborhood is Not assigned

In [109]:
dfNeighborhoods[dfNeighborhoods['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


However, if there were Not assigned neighborhoods, then the following should take care of it:

In [ ]:
dfNeighborhoods['Neighborhood'].loc[(dfNeighborhoods['Neighborhood'] == 'Not assigned')] = dfNeighborhoods['Borough']

dfNeighborhoods.head()

Proof that the replacement would work can be seen in the copied data frame below, where I add a Not assigned neighborhood, and the replace the value with the Borough

In [123]:
dfNeighborhoodsCopy = dfNeighborhoods

row = pd.DataFrame([['M99','Test Borough', 'Not assigned']], columns=['PostalCode', 'Borough', 'Neighborhood'])

dfNeighborhoodsCopy = dfNeighborhoodsCopy.append(row, ignore_index=True)

dfNeighborhoodsCopy.tail()

,PostalCode,Borough,Neighborhood
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
103,M99,Test Borough,Not assigned


In [124]:
dfNeighborhoodsCopy['Neighborhood'].loc[(dfNeighborhoodsCopy['Neighborhood'] == 'Not assigned')] = dfNeighborhoodsCopy['Borough']

dfNeighborhoodsCopy.tail()

,PostalCode,Borough,Neighborhood
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
103,M99,Test Borough,Test Borough


##### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [125]:
dfNeighborhoods.shape

(103, 3)